In [1]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import plotly.express as px
import pmdarima as pm

In [3]:
# Read data from CSV file
df = pd.read_csv('/content/drive/MyDrive/3-Time Series Analysis/0-international-airline-passengers.csv', index_col='Month', parse_dates=True)
df.head()

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [4]:
# Split data into train and test set
n_test = 12
train = df.iloc[: -n_test]
test = df.iloc[-n_test: ]

In [5]:
# Create model
model = pm.auto_arima(train['Passengers'], trace=True, suppress_warnings=True, seasonal=True, m=12)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,1,1)[12] intercept   : AIC=inf, Time=4.14 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=1022.234, Time=0.04 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=906.857, Time=0.44 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=963.555, Time=0.46 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=1192.711, Time=0.04 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=907.180, Time=0.11 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=908.543, Time=1.44 sec
 ARIMA(1,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=2.81 sec
 ARIMA(1,0,0)(0,1,1)[12] intercept   : AIC=907.120, Time=1.24 sec
 ARIMA(1,0,0)(2,1,1)[12] intercept   : AIC=inf, Time=10.19 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=1021.611, Time=0.54 sec
 ARIMA(2,0,0)(1,1,0)[12] intercept   : AIC=906.396, Time=1.40 sec
 ARIMA(2,0,0)(0,1,0)[12] intercept   : AIC=906.075, Time=0.42 sec
 ARIMA(2,0,0)(0,1,1)[12] intercept   : AIC=906.713, Time=1.18 sec
 ARIMA(2,0,0)(1,1,1)[12] intercept   : AI

In [6]:
# Summary
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  132
Model:             SARIMAX(3, 0, 0)x(0, 1, 0, 12)   Log Likelihood                -447.843
Date:                            Mon, 28 Aug 2023   AIC                            905.686
Time:                                    12:02:42   BIC                            919.623
Sample:                                01-01-1949   HQIC                           911.346
                                     - 12-01-1959                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      5.5341      2.007      2.757      0.006       1.600       9.468
ar.L1          0.7049      0.095      7.393      0.000       0.518       0.892
ar.L2          0.2574      0.131      1.968      0.049       0.001       0.514
ar.L3         -0.1434      0.107     -1.338      0.181      -0.354       0.067
sigma2       101.0969     12.818      7.887      0.000      75.974     126.220
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 2.83
Prob(Q):                              0.96   Prob(JB):                         0.24
Heteroskedasticity (H):               1.41   Skew:                            -0.14
Prob(H) (two-sided):                  0.29   Kurtosis:                         3.70
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [7]:
# Prediction
test_pred, conf = model.predict(n_periods=n_test, return_conf_int=True)

In [8]:
# Plot
# Predict test data
# The fill property is an enumeration that may be specified as one of the following enumeration values:
# ['none', 'tozeroy', 'tozerox', 'tonexty', 'tonextx', 'toself', 'tonext'].
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=test.index, y=test['Passengers'], name='Test Passengers'))
fig.add_trace(go.Scatter(x=test.index, y=test_pred, name='Forecast'))
fig.add_trace(go.Scatter(x=test.index, y=conf[:,1], name='Upper'))
fig.add_trace(go.Scatter(x=test.index, y=conf[:,0], fill='tonexty', name='Lower'))
fig.show()

In [9]:
# Predict whole data
train_pred = model.predict_in_sample(start=0, end=-1)
# Plot
# The fill property is an enumeration that may be specified as one of the following enumeration values:
# ['none', 'tozeroy', 'tozerox', 'tonexty', 'tonextx', 'toself', 'tonext'].
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Passengers'], name='Passengers'))
fig.add_trace(go.Scatter(x=train.index, y=train_pred, name='Train Forecast'))
fig.add_trace(go.Scatter(x=test.index, y=test_pred, name='Test Forecast'))
fig.add_trace(go.Scatter(x=test.index, y=conf[:,1], name='Upper'))
fig.add_trace(go.Scatter(x=test.index, y=conf[:,0], fill='tonexty', name='Lower'))
fig.show()